# Typed Objects

In addition to the interfaces offered by `ObjectAPIClient` and `DownloadedObject`, which provides access to Geoscience Objects in a way that is agnostic to the specific object type, the `evo-objects` package also provides a set of "typed objects" that represent specific object types.

As usual, we need to first authenticate, which we can do using the `ServiceManagerWidget` from the `evo-notebooks` package

In [1]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id",
    cache_location="./notebook-data",
).login()

C:\Source\PUBLIC\evo-python-sdk-public\packages\evo-sdk-common\src\evo\notebooks\authorizer.py:108: UserWarning: The evo.notebooks.AuthorizationCodeAuthorizer is not secure, and should only ever be used in Jupyter notebooks in a private environment.
  warnings.warn(


ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

In [2]:
%load_ext evo.widgets

## Working with Variograms

Variograms are geostatistical models that describe spatial correlation structure. They are fundamental to kriging interpolation and resource estimation.

A variogram consists of:
- **Nugget**: The variance at zero lag (y-intercept), representing measurement error or micro-scale variation
- **Sill**: The total variance (plateau value)
- **Structures**: Mathematical models (spherical, exponential, etc.) that define how correlation decreases with distance
- **Anisotropy**: Directional variation in correlation ranges

### Creating a Variogram

To create a variogram, use `VariogramData` with one or more structure types. Common structures include:
- `SphericalStructure` - Most common, reaches sill at finite range
- `ExponentialStructure` - Approaches sill asymptotically
- `GaussianStructure` - Smooth near origin, parabolic behavior
- `CubicStructure` - Smooth transitions, bounded

In [4]:
from evo.objects.typed import (
    Ellipsoid,
    EllipsoidRanges,
    Rotation,
    SphericalStructure,
    Variogram,
    VariogramData,
)

# Define a variogram with two nested structures
variogram_data = VariogramData(
    name="Gold Grade Variogram",
    description="Nested spherical variogram for Au grades",
    sill=1.0,  # Total variance (nugget + all contributions must equal sill)
    nugget=0.1,  # 10% nugget effect
    is_rotation_fixed=True,  # All structures share the same rotation
    modelling_space="data",  # Original units (not normal score transformed)
    data_variance=1.0,  # Variance of the input data
    attribute="grade",  # The attribute this variogram models
    structures=[
        # Short-range structure (30% of variance)
        SphericalStructure(
            contribution=0.3,
            anisotropy=Ellipsoid(
                ranges=EllipsoidRanges(
                    major=15,  # 100m range in major direction
                    semi_major=13,  # 75m in semi-major direction
                    minor=8.5,  # 50m in minor (vertical) direction
                ),
                rotation=Rotation(
                    dip=65,  # Horizontal
                    dip_azimuth=100,  # Strike direction
                    pitch=75,
                ),
            ),
        ),
        # Long-range structure (60% of variance, nugget + 0.3 + 0.6 = 1.0)
        SphericalStructure(
            contribution=0.6,
            anisotropy=Ellipsoid(
                ranges=EllipsoidRanges(
                    major=134,  # 300m range in major direction
                    semi_major=90,  # 200m in semi-major direction
                    minor=40,  # 100m in minor direction
                ),
                rotation=Rotation(
                    dip=65,
                    dip_azimuth=100,
                    pitch=75,
                ),
            ),
        ),
    ],
)

# Create the variogram object
created_variogram = await Variogram.create(manager, variogram_data)
print(f"Created variogram: {created_variogram.metadata.url}")

Created variogram: https://350mt.api.integration.seequent.com/geoscience-object/orgs/829e6621-0ab6-4d7d-96bb-2bb5b407a5fe/workspaces/783b6eef-01b9-42a7-aaf4-35e153e6fcbe/objects/2616ce56-5552-4ba3-b741-08fa32179f32?version=1771271458517602015


In [5]:
created_variogram

### Inspecting a Variogram

The `Variogram` class provides properties to access the variogram model parameters and methods for visualization.

In [6]:
# Load the variogram we just created
variogram = await Variogram.from_reference(manager, created_variogram.metadata.url)

print(f"Variogram: {variogram.name}")
print(f"Sill: {variogram.sill}")
print(f"Nugget: {variogram.nugget}")
print(f"Number of structures: {len(variogram.structures)}")
print(f"Modelling space: {variogram.modelling_space}")
print(f"Attribute: {variogram.attribute}")

# Inspect the structures
for i, struct in enumerate(variogram.structures):
    vtype = struct.get("variogram_type")
    contribution = struct.get("contribution")
    ranges = struct.get("anisotropy", {}).get("ellipsoid_ranges", {})
    print(f"\nStructure {i + 1}: {vtype}")
    print(f"  Contribution: {contribution}")
    print(f"  Ranges: major={ranges.get('major')}, semi_major={ranges.get('semi_major')}, minor={ranges.get('minor')}")

Variogram: Gold Grade Variogram 10
Sill: 1.0
Nugget: 0.1
Number of structures: 2
Modelling space: data
Attribute: grade

Structure 1: spherical
  Contribution: 0.3
  Ranges: major=15, semi_major=13, minor=8.5

Structure 2: spherical
  Contribution: 0.6
  Ranges: major=134, semi_major=90, minor=40


### Getting Variogram Curve Data

The `get_principal_directions()` method returns variogram curves for plotting along the three principal axes.

In [7]:
# Get curve data for the three principal directions
major, semi_major, minor = variogram.get_principal_directions()

print(f"Major direction: range={major.range_value}, sill={major.sill}")
print(f"Semi-major direction: range={semi_major.range_value}")
print(f"Minor direction: range={minor.range_value}")
print(f"Points per curve: {len(major.distance)}")

Major direction: range=134, sill=1.0
Semi-major direction: range=90
Minor direction: range=40
Points per curve: 200


### Visualizing Variograms with Plotly

Use `plotly` to create interactive variogram plots. The variogram curve shows how spatial correlation decreases with distance.

In [9]:
import plotly.graph_objects as go

# Create variogram curve plot
fig = go.Figure()

# Add curves for each principal direction
fig.add_trace(
    go.Scatter(
        x=minor.distance,
        y=minor.semivariance,
        name=f"Minor (range={minor.range_value:.0f}m)",
        line=dict(color="blue", width=2),
    )
)

fig.add_trace(
    go.Scatter(
        x=semi_major.distance,
        y=semi_major.semivariance,
        name=f"Semi-major (range={semi_major.range_value:.0f}m)",
        line=dict(color="green", width=2),
    )
)

fig.add_trace(
    go.Scatter(
        x=major.distance,
        y=major.semivariance,
        name=f"Major (range={major.range_value:.0f}m)",
        line=dict(color="red", width=2),
    )
)

# Add reference lines for nugget and sill
fig.add_hline(
    y=variogram.nugget, line_dash="dash", line_color="gray", annotation_text="Nugget", annotation_position="right"
)
fig.add_hline(
    y=variogram.sill, line_dash="dash", line_color="black", annotation_text="Sill", annotation_position="right"
)

fig.update_layout(
    title="Variogram Model - Principal Directions",
    xaxis_title="Lag Distance (m)",
    yaxis_title="Semivariance γ(h)",
    legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99),
    template="plotly_white",
)

fig.show()

### Variogram in Arbitrary Directions

Use `get_direction()` to calculate the variogram curve in any direction specified by azimuth and dip angles.

In [10]:
# Calculate variogram in an arbitrary direction
distance, semivariance = variogram.get_direction(azimuth=90, dip=30)

fig = go.Figure()

# Add the principal directions for comparison
fig.add_trace(go.Scatter(x=major.distance, y=major.semivariance, name="Major", line=dict(dash="dot")))
fig.add_trace(go.Scatter(x=minor.distance, y=minor.semivariance, name="Minor", line=dict(dash="dot")))

# Add the custom direction
fig.add_trace(
    go.Scatter(
        x=distance,
        y=semivariance,
        name="Az=90°, Dip=30°",
        line=dict(color="purple", width=3),
    )
)

fig.update_layout(
    title="Variogram in Custom Direction",
    xaxis_title="Lag Distance (m)",
    yaxis_title="Semivariance γ(h)",
    template="plotly_white",
)

fig.show()

## Working with Ellipsoids

Ellipsoids represent 3D spatial regions and are used for:
- **Search neighborhoods** in kriging - defining which samples influence each estimated point
- **Variogram anisotropy** - visualizing the directional ranges of spatial correlation
- **Geological domains** - representing oriented geological features

The `Ellipsoid` class provides methods for creating, scaling, and visualizing ellipsoids.

### Creating an Ellipsoid

Create an ellipsoid directly using `Ellipsoid`, `EllipsoidRanges`, and `Rotation`.

In [11]:
from evo.objects.typed import Ellipsoid, EllipsoidRanges, Rotation

# Create an ellipsoid with specified ranges and rotation
ellipsoid = Ellipsoid(
    ranges=EllipsoidRanges(
        major=134,  # 200m in the major direction
        semi_major=90,  # 150m in the semi-major direction
        minor=40,  # 75m in the minor direction
    ),
    rotation=Rotation(
        dip=65,  # Dip angle (degrees)
        dip_azimuth=100,  # Strike direction (degrees from north)
        pitch=75,  # Pitch/rake angle
    ),
)

print(
    f"Ellipsoid ranges: major={ellipsoid.ranges.major}, semi_major={ellipsoid.ranges.semi_major}, minor={ellipsoid.ranges.minor}"
)
print(f"Rotation: azimuth={ellipsoid.rotation.dip_azimuth}°, dip={ellipsoid.rotation.dip}°")

Ellipsoid ranges: major=134, semi_major=90, minor=40
Rotation: azimuth=100°, dip=65°


### Extracting Ellipsoids from Variograms

The `Variogram.get_ellipsoid()` method extracts an ellipsoid from the variogram structures. By default, it selects the structure with the largest volume.

In [12]:
# Get the ellipsoid from the variogram (uses structure with largest volume by default)
var_ellipsoid = variogram.get_ellipsoid()

print("Variogram ellipsoid ranges:")
print(f"  Major: {var_ellipsoid.ranges.major}m")
print(f"  Semi-major: {var_ellipsoid.ranges.semi_major}m")
print(f"  Minor: {var_ellipsoid.ranges.minor}m")

# Or get ellipsoid from a specific structure
first_structure_ellipsoid = variogram.get_ellipsoid(structure_index=0)
print(f"\nFirst structure ranges: major={first_structure_ellipsoid.ranges.major}m")

Variogram ellipsoid ranges:
  Major: 134m
  Semi-major: 90m
  Minor: 40m

First structure ranges: major=15m


### Scaling Ellipsoids for Search Neighborhoods

When using ellipsoids for kriging search neighborhoods, it's common to scale them (typically 2-3x the variogram range).

In [13]:
# Create a search ellipsoid scaled by 2x
search_ellipsoid = var_ellipsoid.scaled(2.0)

print(f"Variogram ellipsoid: major={var_ellipsoid.ranges.major}m")
print(f"Search ellipsoid (2x): major={search_ellipsoid.ranges.major}m")

Variogram ellipsoid: major=134m
Search ellipsoid (2x): major=268.0m


### Visualizing Ellipsoids with Plotly

The `Ellipsoid` class provides methods for 3D visualization:
- `surface_points()` - Generate mesh points for solid rendering
- `wireframe_points()` - Generate wireframe for lighter visualization

In [14]:
import plotly.graph_objects as go

# Define a center point for visualization
center = (500, 500, 100)  # Example coordinates

# Generate surface mesh points
x_surf, y_surf, z_surf = var_ellipsoid.surface_points(center=center, n_points=25)

# Generate wireframe points
x_wire, y_wire, z_wire = var_ellipsoid.wireframe_points(center=center, n_points=50)

# Create 3D figure
fig = go.Figure()

# Add semi-transparent surface mesh
fig.add_trace(
    go.Mesh3d(
        x=x_surf,
        y=y_surf,
        z=z_surf,
        alphahull=0,
        opacity=0.3,
        color="blue",
        name="Variogram Ellipsoid",
    )
)

# Add wireframe for clearer shape definition
fig.add_trace(
    go.Scatter3d(
        x=x_wire,
        y=y_wire,
        z=z_wire,
        mode="lines",
        line=dict(color="darkblue", width=2),
        name="Wireframe",
    )
)

# Add center point
fig.add_trace(
    go.Scatter3d(
        x=[center[0]],
        y=[center[1]],
        z=[center[2]],
        mode="markers",
        marker=dict(size=8, color="red"),
        name="Center",
    )
)

fig.update_layout(
    title="Variogram Anisotropy Ellipsoid",
    scene=dict(
        xaxis_title="Easting (m)",
        yaxis_title="Northing (m)",
        zaxis_title="Elevation (m)",
        aspectmode="data",  # Equal aspect ratio
    ),
    template="plotly_white",
)

fig.show()

### Comparing Variogram and Search Ellipsoids

Visualize both the variogram ellipsoid and a scaled search ellipsoid together.

In [ ]:
# Get the scaled search ellipsoid
search_ellipsoid = var_ellipsoid.scaled(2.0)

# Generate points for both
x_var, y_var, z_var = var_ellipsoid.wireframe_points(center=center, n_points=50)
x_search, y_search, z_search = search_ellipsoid.wireframe_points(center=center, n_points=50)

fig = go.Figure()

# Variogram ellipsoid (range)
fig.add_trace(
    go.Scatter3d(
        x=x_var,
        y=y_var,
        z=z_var,
        mode="lines",
        line=dict(color="blue", width=3),
        name=f"Variogram Range (major={var_ellipsoid.ranges.major:.0f}m)",
    )
)

# Search ellipsoid (2x range)
fig.add_trace(
    go.Scatter3d(
        x=x_search,
        y=y_search,
        z=z_search,
        mode="lines",
        line=dict(color="orange", width=2, dash="dash"),
        name=f"Search (2x, major={search_ellipsoid.ranges.major:.0f}m)",
    )
)

# Center point
fig.add_trace(
    go.Scatter3d(
        x=[center[0]],
        y=[center[1]],
        z=[center[2]],
        mode="markers",
        marker=dict(size=8, color="red"),
        name="Center",
    )
)

fig.update_layout(
    title="Variogram Range vs Search Ellipsoid",
    scene=dict(
        xaxis_title="Easting (m)",
        yaxis_title="Northing (m)",
        zaxis_title="Elevation (m)",
        aspectmode="data",
    ),
    template="plotly_white",
)

fig.show()